In [2]:
#logistic regression
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset, TensorDataset
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

# Load the data
data = pd.read_csv('pseudodata_præoperation.csv')

#make pd to np
X = data.to_numpy()
y = np.random.choice([0, 1], size=len(data))

# Generate a synthetic dataset
X = StandardScaler().fit_transform(X)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)

# Create a dataset
dataset = TensorDataset(X_tensor, y_tensor)

# Split data into holdout set and remaining set
indices = np.arange(len(dataset))
np.random.seed(42)
np.random.shuffle(indices)
holdout_indices = indices[:500]
remaining_indices = indices[500:]

holdout_set = Subset(dataset, holdout_indices)
remaining_set = Subset(dataset, remaining_indices)

# Define the logistic regression model with L2 regularization
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, 1)
    
    def forward(self, x):
        out = torch.sigmoid(self.linear(x))
        return out

input_dim = 575

# Function to train and evaluate the model with L2 regularization
def train_and_evaluate_model(train_loader, val_loader, input_dim, num_epochs=10, lr=0.01, weight_decay=1e-5):
    model = LogisticRegressionModel(input_dim)
    criterion = nn.BCELoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    # Training loop
    for epoch in range(num_epochs):
        model.train()
        for images, labels in train_loader:
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
    # Evaluate on validation set
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = correct / total
    return accuracy, model

# Outer 10-Fold Cross-Validation
outer_kf = KFold(n_splits=10, shuffle=True, random_state=42)
outer_accuracies = []
best_model_params = None
best_outer_accuracy = 0

for outer_train_index, outer_test_index in outer_kf.split(remaining_indices):
    outer_train_subset = Subset(remaining_set, outer_train_index)
    outer_test_subset = Subset(remaining_set, outer_test_index)
    
    # Inner 10-Fold Cross-Validation
    inner_kf = KFold(n_splits=10, shuffle=True, random_state=42)
    inner_accuracies = []

    for inner_train_index, inner_val_index in inner_kf.split(outer_train_index):
        inner_train_indices = np.array(outer_train_index)[inner_train_index]
        inner_val_indices = np.array(outer_train_index)[inner_val_index]
        
        inner_train_subset = Subset(remaining_set, inner_train_indices)
        inner_val_subset = Subset(remaining_set, inner_val_indices)
        
        train_loader = DataLoader(inner_train_subset, batch_size=64, shuffle=True)
        val_loader = DataLoader(inner_val_subset, batch_size=64, shuffle=False)
        
        accuracy, model = train_and_evaluate_model(train_loader, val_loader, input_dim, num_epochs=10, lr=0.01, weight_decay=1e-5)
        inner_accuracies.append(accuracy)
        print(f'Inner Fold Accuracy: {accuracy * 100:.2f}%')

    mean_inner_accuracy = np.mean(inner_accuracies)
    print(f'Outer Fold Mean Inner Accuracy: {mean_inner_accuracy * 100:.2f}%')

    train_loader = DataLoader(outer_train_subset, batch_size=64, shuffle=True)
    test_loader = DataLoader(outer_test_subset, batch_size=64, shuffle=False)
    
    outer_accuracy, outer_model = train_and_evaluate_model(train_loader, test_loader, input_dim, num_epochs=10, lr=0.01, weight_decay=1e-5)
    outer_accuracies.append(outer_accuracy)
    print(f'Outer Fold Accuracy: {outer_accuracy * 100:.2f}%')

    if outer_accuracy > best_outer_accuracy:
        best_outer_accuracy = outer_accuracy
        best_model_params = outer_model.state_dict()

print(f'Mean Outer Accuracy: {np.mean(outer_accuracies) * 100:.2f}%')
print('Best model parameters:', best_model_params)

# Evaluate the final model on the holdout set
final_model = LogisticRegressionModel(input_dim)
final_model.load_state_dict(best_model_params)
holdout_loader = DataLoader(holdout_set, batch_size=64, shuffle=False)

# Evaluate on holdout set
final_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in holdout_loader:
        outputs = final_model(images)
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

holdout_accuracy = correct / total
print(f'Holdout Set Accuracy: {holdout_accuracy * 100:.2f}%')



Inner Fold Accuracy: 46.69%
Inner Fold Accuracy: 51.24%
Inner Fold Accuracy: 45.87%
Inner Fold Accuracy: 44.21%
Inner Fold Accuracy: 46.69%
Inner Fold Accuracy: 47.93%
Inner Fold Accuracy: 54.36%
Inner Fold Accuracy: 43.57%
Inner Fold Accuracy: 47.72%
Inner Fold Accuracy: 49.79%
Outer Fold Mean Inner Accuracy: 47.81%
Outer Fold Accuracy: 50.93%
Inner Fold Accuracy: 42.15%
Inner Fold Accuracy: 46.69%
Inner Fold Accuracy: 55.37%
Inner Fold Accuracy: 52.07%
Inner Fold Accuracy: 47.52%
Inner Fold Accuracy: 50.83%
Inner Fold Accuracy: 50.21%
Inner Fold Accuracy: 54.36%
Inner Fold Accuracy: 48.96%
Inner Fold Accuracy: 50.21%
Outer Fold Mean Inner Accuracy: 49.84%
Outer Fold Accuracy: 49.07%
Inner Fold Accuracy: 48.76%
Inner Fold Accuracy: 51.24%
Inner Fold Accuracy: 43.39%
Inner Fold Accuracy: 54.13%
Inner Fold Accuracy: 47.11%
Inner Fold Accuracy: 48.76%
Inner Fold Accuracy: 48.96%
Inner Fold Accuracy: 49.79%
Inner Fold Accuracy: 47.72%
Inner Fold Accuracy: 47.72%
Outer Fold Mean Inner Accu